## **Function Calling LLMs - Team Project**

In [8]:
import openai
import json
import os

from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("API_KEY")

## Data

## Functions

In [9]:
from registry import Registry
from pprint import pprint

register = Registry()

def to_json(fcn):
    def wrapper(*args, **kwargs):
        result = fcn(*args, **kwargs)
        return json.dumps(result)
    return wrapper

@to_json
@register.register
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return weather_info

pprint(register.registered_functions)

{'get_current_weather': <function get_current_weather at 0x0000026626736FC0>}


In [10]:
FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    }
]

## LLM

In [55]:
from enum import Enum
from typing import Union

class Role(Enum):
    ASSISTANT = "assistant"
    FUNCTION = "function"
    SYSTEM = "system"
    USER = "user"

MODEL_VERSION = "gpt-3.5-turbo-0613" # gpt-4-0613

CONTEXT = {"role": Role.SYSTEM.value, "content": "Answer in one very short sentence."}

In [58]:
def _add_message_to_context(role:Role, content:str, messages:list, function_call:dict=None) -> None:
    if role == Role.ASSISTANT:
        d = {"role": Role.ASSISTANT.value, "content": content}
        if function_call is not None: d |= {"function_call": function}
        messages.append(d)

    if role == Role.USER:
        messages.append({"role": Role.USER.value, "content": content})
    
    if role == Role.FUNCTION:
        messages.append({"role": Role.FUNCTION.value, "name": function_call["name"], "content": content})
        

def send_message(role:Union[Role.USER, Role.FUNCTION], content:str, messages:list, function_call:dict=None) -> dict:
    
    _add_message_to_context(role, content, messages, function_call)
    
    response = openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=messages,
        function_call="auto",
        functions=FUNCTIONS,
    )
    
    response = response["choices"][0]
    response_message = response["message"]["content"]
    
    _add_message_to_context(Role.ASSISTANT, response_message, messages, function_call)
    
    return response


def handle_function(function:dict) -> json:
    function_name, function_args = function["name"], json.loads(function["arguments"])
    
    if function_name in register.registered_functions:
        function_to_call = register[function_name]
        result = function_to_call(**function_args) # return json string
        return result

def run(user_question:str, verbose=False):    
    messages = [CONTEXT]
    
    # Step 1: send the conversation (context) and available functions to GPT
    response = send_message(Role.USER, user_question, messages)
    
    # Step 2: check if GPT wanted to call a function 
    function_call = response.get("function_call")
        
    if function_call:
        if verbose: print("function-call detected...\n")
        # Step 3: call the function
        function_result = handle_function(function_call)
        
        # Step 4: send the info on the function call and function response to GPT
        if verbose: print(function_result + "\n")
        response = send_message(Role.FUNCTION, function_result, messages, function_call)
    
    if verbose:
        print(messages)
    
    response_message = response["message"]["content"]
    return response_message

In [59]:
response = run("What is 1+10?", verbose=True)
print(response)

[{'role': 'system', 'content': 'Answer in one very short sentence.'}, {'role': 'user', 'content': 'What is 1+10?'}, {'role': 'user', 'content': 'The sum of 1 and 10 is 11.'}]
The sum of 1 and 10 is 11.


## Benchmark